## Install TensorTrade

In [1]:
#!python -m pip install -e .. -U

## Setup Data Fetching




In [2]:
import pandas as pd

data = pd.read_csv('btc_data_train.csv')
test_data = pd.read_csv('btc_data_test.csv')

In [3]:
data.head()

,date,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,7/1/2017 11:00,2505.56,2513.38,2495.12,2509.17,287000.32
1,7/1/2017 12:00,2509.17,2512.87,2484.99,2488.43,393142.50
2,7/1/2017 13:00,2488.43,2488.43,2454.40,2454.43,693254.01
3,7/1/2017 14:00,2454.43,2473.93,2450.83,2459.35,712864.80
4,7/1/2017 15:00,2459.35,2475.00,2450.00,2467.83,682105.41


## Create features with the data module

In [4]:
from tensortrade.data import Node, Module, DataFeed, Stream, Select


def rsi(price: Node, period: float):
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Node, fast: float, slow: float, signal: float) -> Node:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[1:]:
    s = Stream(list(data[c])).rename(data[c].name)
    features += [s]

btc_close = Select("BTC:close")(*features)

features += [
    rsi(btc_close, period=24).rename("BTC:rsi"),
    macd(btc_close, fast=10, slow=50, signal=5).rename("BTC:macd"),
]

feed = DataFeed(features)
feed.compile()



features_test = []
for c in test_data.columns[1:]:
    s = Stream(list(test_data[c])).rename(test_data[c].name)
    features_test += [s]

btc_close = Select("BTC:close")(*features_test)

features_test += [
    rsi(btc_close, period=24).rename("BTC:rsi"),
    macd(btc_close, fast=10, slow=50, signal=5).rename("BTC:macd"),
]

feed_test = DataFeed(features_test)
feed_test.compile()

In [5]:
feed.next()
feed.reset()

feed_test.next()
feed_test.reset()

C:\Users\suuser\PycharmProjects\tensortrade\tensortrade\data\stream\node.py:933: RuntimeWarning: invalid value encountered in double_scalars
  v = (w[::-1] * x).sum() / w.sum()


## Setup Trading Environment

In [6]:
from tensortrade.exchanges import Exchange
from tensortrade.exchanges.services.execution.simulated import execute_order
from tensortrade.data import Stream, DataFeed, Module
from tensortrade.instruments import USD, BTC, ETH
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
import copy

coinbase = Exchange("coinbase", service=execute_order)(
    Stream(list(data["BTC:close"])).rename("USD-BTC")
)

test_coinbase = Exchange("coinbase", service=execute_order)(
    Stream(list(test_data["BTC:close"])).rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10000 * USD),
    Wallet(coinbase, 5 * BTC),
])

test_portfolio = Portfolio(USD, [
    Wallet(test_coinbase, 10000 * USD),
    Wallet(test_coinbase, 5 * BTC),
])

env = TradingEnvironment(
    feed=feed,
    portfolio=portfolio,
    use_internal=False,
    action_scheme="simple",
    reward_scheme=RiskAdjustedReturns(return_algorithm = 'sortino', window_size = 24),
    window_size=24
)

test_env = TradingEnvironment(
    feed=feed_test,
    portfolio=test_portfolio,
    use_internal=False,
    action_scheme="simple",
    reward_scheme=RiskAdjustedReturns(return_algorithm = 'sortino', window_size = 24),
    window_size=24
)


## Example Data Feed Observation

Even though this observation contains data from the internal data feed, since `use_internal=False` this data will not be provided as input to the observation history. The data that will be added to observation history of the environment will strictly be the nodes that have been included into the data feed that has been provided as a parameter to the trading environment.

In [7]:
env.feed.next()



{'BTC:open': 2505.56,
 'BTC:high': 2513.38,
 'BTC:low': 2495.12,
 'BTC:close': 2509.17,
 'BTC:volume': 287000.32,
 'BTC:rsi': nan,
 'BTC:macd': -1.0105496686365867e-13,
 'coinbase:/USD-BTC': 2509.17,
 'coinbase:/USD:/free': 10000.0,
 'coinbase:/USD:/locked': 0.0,
 'coinbase:/USD:/total': 10000.0,
 'coinbase:/BTC:/free': 5.0,
 'coinbase:/BTC:/locked': 0.0,
 'coinbase:/BTC:/total': 5.0,
 'coinbase:/BTC:/worth': 12545.85,
 'net_worth': 22545.85}

In [ ]:
import tensortrade.agents.a2c_lstm_agent
from tensortrade.agents.a2c_lstm_agent import A2C_LSTM_Agent
import importlib
importlib.reload(tensortrade.agents.a2c_lstm_agent)

agent = A2C_LSTM_Agent(env, test_env=test_env)

agent.train(n_steps=24*7*10, eps_end = 0.05, debug=False, discount_factor = 0.99, learning_rate=0.0001, batch_size=24*7, train_end=0.3, n_episodes=500, save_path="D:/Users/suuser/Desktop/test_results2")



====      AGENT ID: 1d9e0755-bba4-4bd4-8a5d-e48b131b5050      ====
[10000.00 USD, 5.00000000 BTC]
====      TRAIN EPISODE ID (1/500): 1509e5f0-1185-4271-9fb1-c873186eb501      ====
step 0/1680
[10000.00 USD, 5.00000000 BTC]
None
exchange: 2509.17
step 24/1680
[19752.10 USD, 0.84372481 BTC]
21818.888294575998
exchange: 2449.6
step 48/1680
[1436.82 USD, 8.02078979 BTC]
21507.242291517003
exchange: 2502.3
step 72/1680
[9959.41 USD, 4.50373740 BTC]
21736.368039382
exchange: 2614.93
step 96/1680
[11180.28 USD, 3.87525512 BTC]
21147.668683947202
exchange: 2572.06
step 120/1680
[18583.26 USD, 0.88623591 BTC]
20890.114349011797
exchange: 2602.98
step 144/1680
[14247.44 USD, 2.37134335 BTC]
20308.5936026
exchange: 2556.0
step 168/1680
[1221.99 USD, 7.32321438 BTC]
19590.442468635
exchange: 2508.25
step 192/1680
[18288.34 USD, 0.38553862 BTC]
19272.7511799908
exchange: 2553.34
step 216/1680
[2255.96 USD, 6.55432220 BTC]
18608.207370336
exchange: 2494.88
step 240/1680
[10468.19 USD, 3.00330361 BT

C:\Users\suuser\PycharmProjects\tensortrade\tensortrade\data\stream\node.py:414: RuntimeWarning: divide by zero encountered in true_divide
  return self.op(self.inputs[0].value, self.inputs[1].value)


In [ ]:
import tensortrade.agents.a2c_agent
from tensortrade.agents.a2c_agent import A2CAgent
import importlib
importlib.reload(tensortrade.agents.a2c_agent)

#agent = A2CAgent(env, validation_feed=feed_test)

#agent.train(n_steps=data.shape[0], batch_size=24*7, train_end=0.3, n_episodes=500, save_path="D:/Users/suuser/Desktop/a2c_test_results")


